In [ ]:
import serial
from smbus2 import SMBus
from time import sleep, time
import csv
from datetime import datetime
import math

# === MPU9250 (Accelerometer + Gyroscope) ===
MPU_ADDR = 0x68
bus = SMBus(1)
PWR_MGMT_1   = 0x6B
SMPLRT_DIV   = 0x19
CONFIG       = 0x1A
GYRO_CONFIG  = 0x1B
ACCEL_CONFIG = 0x1C
INT_ENABLE   = 0x38
ACCEL_XOUT_H = 0x3B
GYRO_XOUT_H  = 0x43

# === QMC5883 (Magnetometer) ===
QMC_ADDR = 0x0D
REG_XOUT_LSB = 0x00
REG_CONTROL_1 = 0x09
REG_SET_RESET = 0x0B

# === GPS ===
ser = serial.Serial('/dev/ttyAMA0', 9600, timeout=1)

# === Functions ===
def mpu_init():
    bus.write_byte_data(MPU_ADDR, PWR_MGMT_1, 0x00)
    sleep(0.1)
    bus.write_byte_data(MPU_ADDR, SMPLRT_DIV, 0x07)
    bus.write_byte_data(MPU_ADDR, ACCEL_CONFIG, 0x00)
    bus.write_byte_data(MPU_ADDR, GYRO_CONFIG, 0x00)
    bus.write_byte_data(MPU_ADDR, CONFIG, 0x00)
    bus.write_byte_data(MPU_ADDR, INT_ENABLE, 0x01)

def read_raw_data(addr):
    high = bus.read_byte_data(MPU_ADDR, addr)
    low = bus.read_byte_data(MPU_ADDR, addr+1)
    value = (high << 8) | low
    if value > 32767:
        value -= 65536
    return value

def mpu_calibrate(samples=500):
    ax_sum = ay_sum = az_sum = gx_sum = gy_sum = gz_sum = 0
    for _ in range(samples):
        ax_sum += read_raw_data(ACCEL_XOUT_H)
        ay_sum += read_raw_data(ACCEL_XOUT_H+2)
        az_sum += read_raw_data(ACCEL_XOUT_H+4)
        gx_sum += read_raw_data(GYRO_XOUT_H)
        gy_sum += read_raw_data(GYRO_XOUT_H+2)
        gz_sum += read_raw_data(GYRO_XOUT_H+4)
        sleep(0.002)
    return (
        ax_sum/samples, ay_sum/samples, (az_sum/samples) - 16384,
        gx_sum/samples, gy_sum/samples, gz_sum/samples
    )

def gps_read():
    while True:
        line = ser.readline().decode('ascii', errors='ignore')
        if line.startswith('$GNGGA'):
            parts = line.split(',')
            if len(parts) > 9 and parts[2] and parts[4]:
                try:
                    lat = convert_to_deg(parts[2], parts[3], 'lat')
                    lon = convert_to_deg(parts[4], parts[5], 'lon')
                    alt = float(parts[9])
                    return lat, lon, alt
                except:
                    return None, None, None

def convert_to_deg(raw_val, direction, typ):
    if typ == 'lat':
        deg = int(raw_val[:2])
        min = float(raw_val[2:])
    else:
        deg = int(raw_val[:3])
        min = float(raw_val[3:])
    result = deg + min / 60
    return -result if direction in ['S', 'W'] else result

def qmc_init():
    bus.write_byte_data(QMC_ADDR, REG_SET_RESET, 0x01)
    bus.write_byte_data(QMC_ADDR, REG_CONTROL_1, 0x1D)
    sleep(0.1)

def qmc_read():
    data = bus.read_i2c_block_data(QMC_ADDR, REG_XOUT_LSB, 6)
    x = data[1] << 8 | data[0]
    y = data[3] << 8 | data[2]
    z = data[5] << 8 | data[4]
    x = x if x < 32768 else x - 65536
    y = y if y < 32768 else y - 65536
    z = z if z < 32768 else z - 65536
    return x, y, z

def qmc_calibrate(duration=10):
    print("Calibrating QMC5883... Move sensor in all directions.")
    start = time()
    data = []
    while time() - start < duration:
        data.append(qmc_read())
        sleep(0.05)
    xs, ys, zs = zip(*data)
    return (sum(xs)/len(xs), sum(ys)/len(ys), sum(zs)/len(zs))

# === Initialization ===
mpu_init()
ax_off, ay_off, az_off, gx_off, gy_off, gz_off = mpu_calibrate()
qmc_init()
x_off, y_off, z_off = qmc_calibrate()

filename = f"imu_gps_mag_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Time', 'Lat', 'Lon', 'Alt (m)',
                     'Gx (Â°/s)', 'Gy (Â°/s)', 'Gz (Â°/s)',
                     'Ax (m/sÂ²)', 'Ay (m/sÂ²)', 'Az (m/sÂ²)',
                     'Mx (ÂµT)', 'My (ÂµT)', 'Mz (ÂµT)', '|M| (ÂµT)'])

    print("Logging started. Press Ctrl+C to stop.\n")
    try:
        while True:
            # GPS
            lat, lon, alt = gps_read()

            # IMU
            ax = (read_raw_data(ACCEL_XOUT_H) - ax_off) / 16384.0 * 9.81
            ay = (read_raw_data(ACCEL_XOUT_H+2) - ay_off) / 16384.0 * 9.81
            az = (read_raw_data(ACCEL_XOUT_H+4) - az_off) / 16384.0 * 9.81
            gx = (read_raw_data(GYRO_XOUT_H) - gx_off) / 131.0
            gy = (read_raw_data(GYRO_XOUT_H+2) - gy_off) / 131.0
            gz = (read_raw_data(GYRO_XOUT_H+4) - gz_off) / 131.0

            # Magnetometer
            mx_raw, my_raw, mz_raw = qmc_read()
            mx = (mx_raw - x_off) * (8.0 / 32768.0) * 100
            my = (my_raw - y_off) * (8.0 / 32768.0) * 100
            mz = (mz_raw - z_off) * (8.0 / 32768.0) * 100
            mag = math.sqrt(mx*2 + my2 + mz*2)

            # Timestamp
            now = datetime.now()
            date_now = now.strftime('%Y-%m-%d')
            time_now = now.strftime('%H:%M:%S')

            # Log
            print(f"[{time_now}] Lat:{lat:.6f}, Lon:{lon:.6f}, Alt:{alt:.2f}m | "
                  f"Gx:{gx:.2f}, Gy:{gy:.2f}, Gz:{gz:.2f} | "
                  f"Ax:{ax:.2f}, Ay:{ay:.2f}, Az:{az:.2f} | "
                  f"Mx:{mx:.2f}, My:{my:.2f}, Mz:{mz:.2f}, |M|:{mag:.2f}")

            writer.writerow([date_now, time_now, lat, lon, alt,
                             gx, gy, gz, ax, ay, az,
                             mx, my, mz, mag])
            file.flush()
            sleep(1)

    except KeyboardInterrupt:
        print("\nLogging stopped by user.")
        ser.close()